# Lab 2


In [1]:
import pandas as pd
import duckdb
from sklearn.linear_model import LinearRegression
import numpy as np
from scipy.optimize import minimize

In [2]:
# K - Обсяг витрат капіталу
# L - Обсяг витрат трудових ресурсів
# F - Обсяг випуску продукції

column_names = ["K", "L", "F"]

data_manual = [
    [5700, 12245, 140330],  # 1
    [4740, 13340, 120355],  # 2
    [4390, 13860, 125000],  # 3
    [5330, 14400, 137330],  # 4
    [5200, 14000, 121570],  # 5
    [4160, 11000, 113100],  # 6
    [5200, 14145, 133000],  # 7
    [4690, 13900, 126165],  # 8
    [5890, 15050, 149000],  # 9
    [4930, 13060, 120950]   # 10
]


df = pd.DataFrame(data_manual, columns=column_names)

df

,K,L,F
0,5700,12245,140330
1,4740,13340,120355
2,4390,13860,125000
3,5330,14400,137330
4,5200,14000,121570
5,4160,11000,113100
6,5200,14145,133000
7,4690,13900,126165
8,5890,15050,149000
9,4930,13060,120950


In [3]:
# F = A * K^alpha * L^beta
# ln(F) = ln(A) + alpha * ln(K) + beta * ln(L)

ln_df = duckdb.sql("SELECT ln(F) as ln_F, ln(K) as ln_K, ln(L) as ln_L FROM df").to_df()

model = LinearRegression().fit(ln_df[["ln_K", "ln_L"]], ln_df["ln_F"])

alpha, beta, A = *model.coef_, np.exp(model.intercept_)

print(f"F = {A} * K^{alpha} * L^{beta}")

def production_function(K, L):
    return A * np.power(K, alpha) * np.power(L, beta)

F = 180.57693837785925 * K^0.5901866073617685 * L^0.16193259000760254


In [4]:
df_copy = df.copy()
df_copy["F'"] = production_function(df["K"], df["L"])
df_copy

,K,L,F,F'
0,5700,12245,140330,136553.576338
1,4740,13340,120355,124180.842504
2,4390,13860,125000,119421.496779
3,5330,14400,137330,134741.439261
4,5200,14000,121570,132187.668629
5,4160,11000,113100,111438.660886
6,5200,14145,133000,132408.412249
7,4690,13900,126165,124230.561327
8,5890,15050,149000,143950.489462
9,4930,13060,120950,126659.106256


In [5]:

a_coef = alpha + beta

print(f"{a_coef} < 1")
print("дохід від розширення масштабу є спадним")


MRTS_coef = alpha / beta
print(MRTS_coef)

print("Еластичність заміщення для мультиплікативної функції = 1")

0.752119197369371 < 1
Сталий дохід від розширення масштабу є спадним
3.644643782539759
Еластичність заміщення для мультиплікативної функції = 1


In [ ]:
"""
MP_K = dF/dK
MP_L = dF/dL

MP_K/MP_L = theta
L / K = theta / MRTS_coef
sigma = (d(ln(L/K)) / d(ln(MP_K/MP_L)) = d(L/K) / L/K * theta / d(theta) = d(L/K) / d(theta) * MRTS_coef = d(theta) / d(theta) = 1
"""

In [6]:
# K - Обсяг витрат капіталу
# L - Обсяг витрат трудових ресурсів
# F - Обсяг випуску продукції

# Constants
PRICE = 200
WAGE_CAPITAL = 150 # w_k
WAGE_LABOR = 50 # w_l

# Initial guess for the optimization algorithm
initial_guess = [1, 1]

def profit(x):
    # Calculate the negative profit to be minimized
    K, L = x
    return -(PRICE * production_function(K, L) - WAGE_CAPITAL * K - WAGE_LABOR * L)

def optimize_profit(constraints=None):
    # Optimize profit using the SLSQP method with optional constraints
    result = minimize(profit, initial_guess, bounds=[(0, None), (0, None)], method="SLSQP", constraints=constraints)

    if result.success:
        optimized_K, optimized_L = result.x
        return f"Оптимальний K: {optimized_K:.2f}, Оптимальний L: {optimized_L:.2f}\nМаксимальний прибуток: {-result.fun:.2f}"
    else:
        return "Оптимізація не вдалася."

# Long-term period optimization
print("Довгостроково:")
print(optimize_profit())
print()
# Short-term period optimization with constraints
constraint = [{"type": "ineq", "fun": lambda x: -(x[0] + 2*x[1] - 100)}]
print("Короткостроково:")
print(optimize_profit(constraints=constraint))


Довгостроково:
Оптимальний K: 134012525.47, Оптимальний L: 112246860.84
Максимальний прибуток: 19764704769.22

Короткостроково:
Оптимальний K: 78.06, Оптимальний L: 10.97
Максимальний прибуток: 684403.69


In [19]:
def p_func(q):
    return 1000 - 0.5 * q

def w_K_func(q):
    return 2 * q + 10

def w_L_func(q):
    return 3 * q + 15

def profit_monopoly(x):
    # Calculates the negative profit for a monopoly-monopsony scenario.
    K, L = x
    q = production_function(K, L)
    return - (p_func(q) * q - K * w_K_func(K) - L * w_L_func(L))

def optimize_monopoly_profit():
    initial_guess = [10, 10]
    result = minimize(profit_monopoly, initial_guess, bounds=[(0, None), (0, None)], method="SLSQP")

    if result.success:
        optimized_K, optimized_L = result.x
        q = production_function(optimized_K, optimized_L)
        print(f"Оптимальний K: {optimized_K:.2f}, Оптимальний L: {optimized_L:.2f}")
        print(f"Максимальний прибуток: {-result.fun:.2f}")
        print(f"Ціна продукції: {p_func(q):.2f}")
        print(f"Ціна капіталу: {w_K_func(q):.2f}")
        print(f"Ціна праці: {w_L_func(q):.2f}")
    else:
        print("Оптимізація не вдалася.")

# Run the monopoly-monopsony optimization
optimize_monopoly_profit()


Оптимальний K: 11.99, Оптимальний L: 4.52
Максимальний прибуток: 499462.71
Ціна продукції: 500.59
Ціна капіталу: 2007.64
Ціна праці: 3011.46
